# 🎨 딥러닝 2일차 복습: Vanilla GAN 완전 정복

이번 복습에서는 2일차에 학습한 **Vanilla GAN**의 핵심 개념들을 다시 정리하고, 실습 문제를 통해 이해도를 점검합니다.

## 📚 학습 목표
- GAN의 핵심 개념(Min-Max Game)과 학습 원리 이해
- Generator와 Discriminator의 역할과 관계 파악
- Label Smoothing과 하이퍼파라미터 튜닝 능력 향상
- PyTorch로 GAN 모델 직접 구현 및 학습 루프 완성

---

## ⚙️ 환경 설정

In [ ]:
# 필요한 라이브러리 임포트
import torch
import torch.nn as nn
import torch.optim as optim
import torch.nn.functional as F
from torch.utils.data import DataLoader
from torchvision import datasets, transforms
import numpy as np
import matplotlib.pyplot as plt

# 한글 폰트 설정 (Colab 등 환경에 따라 다를 수 있음)
plt.rc('font', family='NanumGothic')
plt.rcParams['axes.unicode_minus'] = False

# 시드 고정
torch.manual_seed(42)
np.random.seed(42)

# GPU 확인
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
print(f"사용 디바이스: {device}")
if torch.cuda.is_available():
    print(f"GPU 이름: {torch.cuda.get_device_name(0)}")

---
# Part 1: 객관식 퀴즈 🧠

먼저 2일차에 학습한 내용을 객관식 문제로 점검해봅시다.

## 퀴즈 1: GAN의 기본 구조



![Image of GAN architecture diagram]


**문제:** 위 다이어그램과 같이 GAN(Generative Adversarial Network)을 구성하는 두 개의 핵심 네트워크는 무엇인가요?

1. Encoder와 Decoder
2. Generator와 Discriminator
3. Encoder와 Classifier
4. Generator와 Classifier

<details>
<summary><b>정답 및 풀이 보기</b></summary>

**정답: 2번**

**풀이:**
- GAN은 **Generator(생성자, G)**와 **Discriminator(판별자, D)** 두 개의 신경망으로 구성됩니다.
- **Generator의 역할**: 랜덤 노이즈 벡터 z를 입력받아 가짜 데이터를 생성하며, 목표는 Discriminator를 속이는 것입니다.
- **Discriminator의 역할**: 이미지를 입력받아 진짜/가짜를 판별(0~1 사이 확률)하며, 목표는 정확히 구별하는 것입니다.

</details>

## 퀴즈 2: GAN의 학습 순서

**문제:** 일반적인 GAN의 학습 순서로 올바른 것은?

1. Generator와 Discriminator를 동시에 학습
2. Discriminator만 먼저 학습 후 Generator 학습
3. Discriminator 학습 → Generator 학습을 번갈아 반복
4. Generator만 먼저 학습 후 Discriminator 학습

<details>
<summary><b>정답 및 풀이 보기</b></summary>

**정답: 3번**

**풀이:**
- GAN은 한 배치 안에서 다음 순서로 학습합니다:
  1. **Discriminator 학습** (G 고정): 진짜/가짜 구분 능력 향상
  2. **Generator 학습** (D 고정): D를 속이는 능력 향상
- 이 과정을 반복하며 두 모델이 **균형있게 발전**하는 것이 중요합니다.

</details>

## 퀴즈 3: 활성화 함수 선택

**문제:** Vanilla GAN에서 Generator의 **출력층**과 Discriminator의 **출력층**에 주로 사용하는 활성화 함수 조합은? (이미지는 [-1, 1]로 정규화됨)

1. G: ReLU / D: Softmax
2. G: Sigmoid / D: Tanh
3. G: Tanh / D: Sigmoid
4. G: Linear / D: ReLU

<details>
<summary><b>정답 및 풀이 보기</b></summary>

**정답: 3번**

**풀이:**
- **Generator 출력**: 이미지가 `[-1, 1]` 범위이므로 **Tanh**를 사용합니다.
- **Discriminator 출력**: 진짜일 확률(0~1)을 출력해야 하므로 **Sigmoid**를 사용합니다.

</details>

## 퀴즈 4: 손실 함수 (Loss Function)

**문제:** GAN에서 일반적으로 사용하는 손실 함수는?

1. Mean Squared Error (MSE)
2. Cross Entropy Loss
3. Binary Cross Entropy (BCE)
4. Kullback-Leibler Divergence

<details>
<summary><b>정답 및 풀이 보기</b></summary>

**정답: 3번**

**풀이:**
- GAN은 진짜(1)와 가짜(0)를 구분하는 **이진 분류(Binary Classification)** 문제 기반이므로 **BCE**를 사용합니다.
- 수식: $\text{BCE}(y, \hat{y}) = -[y \log(\hat{y}) + (1-y)\log(1-\hat{y})]$

</details>

## 퀴즈 5: Mode Collapse

**문제:** Mode Collapse란 무엇인가요?

1. Discriminator가 모든 이미지를 진짜로 판단하는 현상
2. Generator가 한 가지 패턴만 반복 생성하는 현상
3. 학습이 너무 빠르게 수렴하는 현상
4. Loss가 발산하는 현상

<details>
<summary><b>정답 및 풀이 보기</b></summary>

**정답: 2번**

**풀이:**
- **Mode Collapse**는 Generator가 다양한 데이터를 생성하지 못하고, Discriminator를 쉽게 속일 수 있는 **특정 패턴(예: 숫자 '8')만 계속 생성**하는 현상입니다.
- 다양성(Diversity)이 상실된 실패 사례입니다.

</details>

---
# Part 2: 서술형 문제 (핵심 개념) 📝

핵심 개념들을 자신의 언어로 설명해보세요.

## 문제 1: GAN의 학습 원리 (Min-Max Game)

**문제:** GAN의 학습 목표를 나타내는 아래 수식의 의미를 Discriminator($D$)와 Generator($G$)의 관점에서 각각 설명하세요.

$$\min_G \max_D V(D, G) = \mathbb{E}_{x \sim p_{data}}[\log D(x)] + \mathbb{E}_{z \sim p_z}[\log(1 - D(G(z)))]$$

---

### 여러분의 답안:
```
[여기에 답안을 작성하세요]


```

<details>
<summary><b>모범 답안 보기</b></summary>

**Discriminator ($D$)의 관점 (Maximize):**
- $D$는 위 수식 전체를 **최대화**하려고 합니다.
- $\log D(x)$ 최대화: 진짜 데이터를 진짜(1)라고 정확히 판별.
- $\log(1 - D(G(z)))$ 최대화: 가짜 데이터를 가짜(0)라고 정확히 판별.

**Generator ($G$)의 관점 (Minimize):**
- $G$는 위 수식 전체를 **최소화**하려고 합니다.
- 특히 두 번째 항 $\log(1 - D(G(z)))$를 최소화하여, $D(G(z))$가 1(진짜)에 가까워지도록, 즉 **Discriminator를 속이는 것**이 목표입니다.

</details>

## 문제 2: Label Smoothing

**문제:** Label Smoothing 기법(예: 진짜 라벨을 1.0 대신 0.9로 설정)이 GAN 학습에서 어떤 효과가 있는지 서술하세요.

---

### 여러분의 답안:
```
[여기에 답안을 작성하세요]


```

<details>
<summary><b>모범 답안 보기</b></summary>

**효과:**
1. **Overconfidence 방지**: Discriminator가 너무 강하게 확신(0 또는 1)하는 것을 막아줍니다.
2. **Gradient 소멸 방지**: D가 너무 완벽해지면 G에게 전달되는 Gradient가 사라지는데, Label Smoothing은 D가 항상 약간의 불확실성을 갖게 하여 G가 계속 학습할 수 있게 돕습니다.
3. **Mode Collapse 완화**: 결과적으로 학습 과정을 안정화시켜 다양한 이미지를 생성하도록 돕습니다.

</details>

---
# Part 3: Shape 추적 문제 📐

GAN 구현에서 가장 헷갈리는 텐서의 Shape 흐름을 완벽히 이해해봅시다.

## 문제: 배치 크기 128일 때 Shape 추적

**입력:**
- batch_size = 128
- z_dim = 100 (노이즈 차원)
- 이미지 크기 = 28×28 = 784

**각 단계의 텐서 shape을 채우세요:**

1. noise (입력 노이즈): ( ____ , ____ )
2. fake_images (Generator 출력): ( ____ , ____ )
3. real_output (Discriminator 출력): ( ____ , ____ )
4. d_loss (최종 손실값): 스칼라 (Scalar)

---

<details>
<summary><b>정답 보기</b></summary>

1. **noise**: (128, 100)
2. **fake_images**: (128, 784)
3. **real_output**: (128, 1) - 각 이미지당 하나의 확률값
4. **d_loss**: 스칼라 - 배치 전체의 평균 Loss

</details>

## 실습: Shape 검증 코드

아래 코드를 실행하여 정답을 확인해보세요.

In [ ]:
# 간단한 Generator/Discriminator 정의
class SimpleGenerator(nn.Module):
    def __init__(self, z_dim=100):
        super().__init__()
        self.net = nn.Sequential(
            nn.Linear(z_dim, 256),
            nn.ReLU(),
            nn.Linear(256, 784),
            nn.Tanh()
        )
    def forward(self, z): return self.net(z)

class SimpleDiscriminator(nn.Module):
    def __init__(self):
        super().__init__()
        self.net = nn.Sequential(
            nn.Linear(784, 256),
            nn.ReLU(),
            nn.Linear(256, 1),
            nn.Sigmoid()
        )
    def forward(self, x): return self.net(x)

# 텐서 생성 및 Shape 확인
batch_size = 128
z_dim = 100
generator = SimpleGenerator(z_dim)
discriminator = SimpleDiscriminator()

noise = torch.randn(batch_size, z_dim)
print(f"1. noise shape: {noise.shape}")

fake_images = generator(noise)
print(f"2. fake_images shape: {fake_images.shape}")

real_output = discriminator(torch.randn(batch_size, 784))
print(f"3. real_output shape: {real_output.shape}")

---
# Part 4: 코드 구현 문제 (PyTorch) 💻

이제 실제 학습 코드를 완성해봅시다.

## 문제 1: 모델 클래스 구현

빈칸을 채워 Generator와 Discriminator를 완성하세요.
- **Generator**: Linear -> ReLU -> Linear -> ReLU -> Linear -> **Tanh**
- **Discriminator**: Linear -> LeakyReLU -> Linear -> LeakyReLU -> Linear -> **Sigmoid**

In [ ]:
# TODO: Generator 클래스를 완성하세요
class Generator(nn.Module):
    def __init__(self, z_dim=100, img_dim=784):
        super(Generator, self).__init__()
        self.net = nn.Sequential(
            nn.Linear(z_dim, 256),
            nn.ReLU(True),
            nn.Linear(256, 512),
            nn.ReLU(True),
            nn.Linear(512, img_dim),
            # TODO: 마지막 활성화 함수는?
            # 코드 작성:
        )

    def forward(self, z):
        return self.net(z)

# TODO: Discriminator 클래스를 완성하세요
class Discriminator(nn.Module):
    def __init__(self, img_dim=784):
        super(Discriminator, self).__init__()
        self.net = nn.Sequential(
            nn.Linear(img_dim, 512),
            nn.LeakyReLU(0.2, inplace=True),
            nn.Linear(512, 256),
            nn.LeakyReLU(0.2, inplace=True),
            nn.Linear(256, 1),
            # TODO: 마지막 활성화 함수는?
            # 코드 작성:
        )

    def forward(self, x):
        return self.net(x)

<details>
<summary><b>모범 답안 보기</b></summary>

```python
# Generator 마지막 활성화 함수
nn.Tanh()

# Discriminator 마지막 활성화 함수
nn.Sigmoid()
```
</details>

## 문제 2: 학습 루프 완성하기 (핵심!)

GAN 학습의 가장 중요한 부분인 `train_step` 함수를 완성하세요.
- `detach()` 사용 위치에 주의하세요.
- 라벨(1, 0)을 올바르게 설정하세요.

In [ ]:
# 모델 초기화
G = Generator(z_dim=100).to(device)
D = Discriminator(img_dim=784).to(device)
criterion = nn.BCELoss()
opt_g = optim.Adam(G.parameters(), lr=0.0002)
opt_d = optim.Adam(D.parameters(), lr=0.0002)

def train_step(real_images):
    """GAN의 한 스텝 학습"""
    batch_size = real_images.size(0)
    real_images = real_images.to(device)

    # 라벨 생성 (진짜=1, 가짜=0)
    real_labels = torch.ones(batch_size, 1).to(device)
    fake_labels = torch.zeros(batch_size, 1).to(device)

    # ----------------------------
    # 1. Discriminator 학습
    # ----------------------------
    opt_d.zero_grad()

    # 1-1. 진짜 이미지 Loss
    outputs_real = D(real_images)
    d_loss_real = criterion(outputs_real, real_labels)

    # 1-2. 가짜 이미지 Loss
    z = torch.randn(batch_size, 100).to(device)
    fake_images = G(z)

    # TODO: fake_images를 D에 넣을 때 주의할 점은? (힌트: detach)
    outputs_fake = # 여기에 작성: D(...)
    d_loss_fake = criterion(outputs_fake, fake_labels)

    # 역전파
    d_loss = d_loss_real + d_loss_fake
    d_loss.backward()
    opt_d.step()

    # ----------------------------
    # 2. Generator 학습
    # ----------------------------
    opt_g.zero_grad()

    # 새로운 노이즈 생성
    z = torch.randn(batch_size, 100).to(device)
    fake_images = G(z)

    # TODO: G 학습 시 D를 속여야 합니다. Target Label은 무엇일까요?
    outputs = D(fake_images) # detach 없음!
    g_loss = # 여기에 작성: criterion(...)

    # 역전파
    g_loss.backward()
    opt_g.step()

    return d_loss.item(), g_loss.item()

<details>
<summary><b>모범 답안 보기</b></summary>

```python
# Discriminator 학습 시 가짜 이미지 처리
outputs_fake = D(fake_images.detach())

# Generator 학습 시 Loss 계산
# 목표: D가 '진짜(real_labels)'라고 판단하게 만드는 것
g_loss = criterion(outputs, real_labels)
```

**핵심 포인트:**
- `detach()`: D를 학습할 때는 G로 gradient가 흐르지 않게 끊어줘야 합니다.
- G 학습 목표: `fake_images`에 대해 D가 `1 (real_labels)`을 출력하도록 학습합니다.

</details>

## 문제 3: Label Smoothing 적용

위 `train_step` 함수에서 **Label Smoothing**을 적용하려면 코드를 어떻게 수정해야 할까요?

**수정할 부분:**
```python
# 기존:
real_labels = torch.ones(batch_size, 1).to(device)

# 수정 후 (0.9로 설정):
real_labels = # 코드를 작성하세요
```

<details>
<summary><b>정답 보기</b></summary>

```python
real_labels = torch.ones(batch_size, 1).to(device) * 0.9
```
이렇게 하면 진짜 이미지의 타겟값이 1.0에서 0.9로 변경되어, D가 과도하게 확신하는 것을 방지합니다.
</details>

---
# Part 5: 미니 프로젝트 - MNIST GAN 완성 🎨

배운 내용을 종합하여 실제 MNIST 데이터를 생성하는 전체 파이프라인을 실행해봅니다.

In [ ]:
# 1. 데이터 로드
transform = transforms.Compose([
    transforms.ToTensor(),
    transforms.Normalize((0.5,), (0.5,)) # [-1, 1] 정규화
])
dataset = datasets.MNIST(root='./data', train=True, transform=transform, download=True)
dataloader = DataLoader(dataset, batch_size=128, shuffle=True)

# 2. 모델 재설정
G = Generator().to(device)
D = Discriminator().to(device)
opt_g = optim.Adam(G.parameters(), lr=0.0002)
opt_d = optim.Adam(D.parameters(), lr=0.0002)
criterion = nn.BCELoss()

# 시각화용 고정 노이즈
fixed_noise = torch.randn(16, 100).to(device)

print("🚀 학습 시작! (약 5 에폭만 진행합니다)")

# 3. 학습 루프
num_epochs = 5
for epoch in range(num_epochs):
    for i, (images, _) in enumerate(dataloader):
        # Flatten input (28x28 -> 784)
        images = images.view(images.size(0), -1)

        # 학습 함수 호출 (위에서 정의한 train_step 사용)
        # 주의: 위 셀의 train_step을 모범 답안으로 채운 후 실행해야 합니다.
        # 여기서는 전체 로직을 다시 간단히 씁니다 for self-contained run

        real_images = images.to(device)
        batch_size = real_images.size(0)

        # Labels
        real_labels = torch.ones(batch_size, 1).to(device)
        fake_labels = torch.zeros(batch_size, 1).to(device)

        # Train D
        opt_d.zero_grad()
        outputs_real = D(real_images)
        d_loss_real = criterion(outputs_real, real_labels)

        z = torch.randn(batch_size, 100).to(device)
        fake_images = G(z)
        outputs_fake = D(fake_images.detach())
        d_loss_fake = criterion(outputs_fake, fake_labels)

        d_loss = d_loss_real + d_loss_fake
        d_loss.backward()
        opt_d.step()

        # Train G
        opt_g.zero_grad()
        z = torch.randn(batch_size, 100).to(device)
        fake_images = G(z)
        outputs = D(fake_images)
        g_loss = criterion(outputs, real_labels)
        g_loss.backward()
        opt_g.step()

    # 에폭마다 결과 출력
    print(f"Epoch [{epoch+1}/{num_epochs}] D Loss: {d_loss.item():.4f}, G Loss: {g_loss.item():.4f}")

    # 이미지 시각화
    with torch.no_grad():
        fake = G(fixed_noise).reshape(-1, 28, 28).cpu()
        fig, axes = plt.subplots(1, 4, figsize=(10, 3))
        for j in range(4):
            axes[j].imshow(fake[j], cmap='gray')
            axes[j].axis('off')
        plt.show()

print("🎉 학습 완료!")

---
## 📝 요약 및 다음 단계

**오늘 배운 핵심:**
1. **GAN 구조**: Generator(위조범) vs Discriminator(경찰)의 경쟁
2. **학습 원리**: Min-Max Game
3. **구현 팁**: `detach()` 사용, `BCE` Loss, `Tanh`/`Sigmoid` 활성화 함수
4. **안정화**: Label Smoothing, 올바른 Loss 사용